This pipeline retrieves significant DEGs and perform appropriate filters to generate biologically relevant expression changing gene sets.

Filter set is:
- log2 FC > 1.
- min max expression > 2 TPM
- noise threshold > 0.5


In [1]:
import pandas, numpy

# 0. user-defined variables

In [2]:
DEG_folder = '/home/adrian/projects/reynisfjara/results/DEGs_DESeq2/'
expression_file = '/home/adrian/projects/reynisfjara/results/tpm/DESeq2_TPM_values.tsv'
annotation_file = '/home/adrian/projects/reynisfjara/results/annotation/annotation.csv'
results_table_dir = '/home/adrian/projects/reynisfjara/results/relevant_DEGs/'

mice = ['a3922', 'a4774', 'a4775', 'a4776']
trends = ['up', 'down']

expression_threshold = 2
discrete_fc_threshold = 1
noise_threshold = 1/2

# 1. read data

## 1.1. expression data

In [3]:
expression = pandas.read_csv(expression_file, sep='\t', index_col=0)
expression.head()

,a3922_0h_1,a3922_0h_2,a3922_0h_3,a3922_48h_1,a3922_48h_2,a3922_48h_3,a3922_72h_1,a3922_72h_2,a3922_72h_3,a4774_0h_1,...,a4775_72h_3,a4776_0h_1,a4776_0h_2,a4776_0h_3,a4776_48h_1,a4776_48h_2,a4776_48h_3,a4776_72h_1,a4776_72h_2,a4776_72h_3
ENSMUSG00000000001,65.244411,65.953285,64.396929,75.289955,71.147817,72.146832,67.786762,70.217188,70.219265,61.227221,...,68.440778,64.806231,65.619286,66.749396,64.425407,68.665136,68.419059,69.705546,70.717428,71.017699
ENSMUSG00000000003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000000028,5.945436,5.464240,4.924612,20.245428,19.780706,22.747363,19.239938,22.062649,20.547492,7.843197,...,14.565935,4.712688,3.690099,4.120889,7.220914,8.377850,9.611563,9.465468,10.235803,9.808747
ENSMUSG00000000037,0.220972,0.959207,0.258270,0.969948,1.149452,2.212842,0.989879,2.354492,1.813624,0.545637,...,0.989086,0.451844,0.523639,0.679725,2.862086,0.865126,1.973570,1.612622,2.148935,5.445061
ENSMUSG00000000049,0.061451,0.061879,0.096945,0.000000,0.071373,0.000000,0.059768,0.000000,0.000000,0.214027,...,0.761343,0.000000,0.125876,0.000000,0.000000,0.096454,0.130578,0.000000,0.000000,0.000000


## 1.2. annotation

In [4]:
annotation = pandas.read_csv(annotation_file, sep=',', index_col='ens_gene')
annotation.drop(columns=['Unnamed: 0', 'target_id'], inplace=True)
annotation.drop_duplicates(inplace=True)
print(annotation.shape)
annotation.head()

(53193, 3)


,gene_biotype,description,ext_gene
ens_gene,,,
ENSMUSG00000064336,Mt_tRNA,mitochondrially encoded tRNA phenylalanine [So...,mt-Tf
ENSMUSG00000064337,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:MGI S...,mt-Rnr1
ENSMUSG00000064338,Mt_tRNA,mitochondrially encoded tRNA valine [Source:MG...,mt-Tv
ENSMUSG00000064339,Mt_rRNA,mitochondrially encoded 16S rRNA [Source:MGI S...,mt-Rnr2
ENSMUSG00000064340,Mt_tRNA,mitochondrially encoded tRNA leucine 1 [Source...,mt-Tl1


In [5]:
annotation.head()

,gene_biotype,description,ext_gene
ens_gene,,,
ENSMUSG00000064336,Mt_tRNA,mitochondrially encoded tRNA phenylalanine [So...,mt-Tf
ENSMUSG00000064337,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:MGI S...,mt-Rnr1
ENSMUSG00000064338,Mt_tRNA,mitochondrially encoded tRNA valine [Source:MG...,mt-Tv
ENSMUSG00000064339,Mt_rRNA,mitochondrially encoded 16S rRNA [Source:MGI S...,mt-Rnr2
ENSMUSG00000064340,Mt_tRNA,mitochondrially encoded tRNA leucine 1 [Source...,mt-Tl1


# 2. define gene sets

In [6]:
for mouse in mice:
    condition_labels0 = ['{}_0h_{}'.format(mouse, i+1) for i in range(3)]
    condition_labels48 = ['{}_48h_{}'.format(mouse, i+1) for i in range(3)]
    condition_labels72 = ['{}_72h_{}'.format(mouse, i+1) for i in range(3)]
    
    for trend in trends:
        print('working with mouse {} and trend {}'.format(mouse, trend))
        
        container = []
        
        DEG_file = DEG_folder + mouse + '_' + trend + '.tsv'
        DEGs = pandas.read_csv(DEG_file, sep='\t', index_col=0)
        print('\t detected {} DEGs'.format(DEGs.shape[0]))
        
        rules = (DEGs['pvalue'] < 0.05) & (DEGs['padj'] < 0.1)
        sub = DEGs[rules]
        print('\t passed tests {}'.format(DEGs.shape[0]))
        
        for ensembl in DEGs.index:
            
            gene_name = ''; biotype = ''; description = ''
            try:
                gene_name = annotation.loc[ensembl]['ext_gene']
                biotype = annotation.loc[ensembl]['gene_biotype']
                description = annotation.loc[ensembl]['description'].split(' [')[0]
            except:
                pass
            
            print('\t\t working with {}'.format(gene_name))
            including = True
            
        
            #
            # filter 1 --- expression
            #
            tpms0 = []
            for label in condition_labels0:
                if label in expression.keys():
                    tpms0.append(expression[label][ensembl])
            tpm0 = numpy.median(tpms0)
            tpms48 = []
            for label in condition_labels48:
                if label in expression.keys():
                    tpms48.append(expression[label][ensembl])
            tpm48 = numpy.median(tpms48)
            tpms72 = []
            for label in condition_labels72:
                if label in expression.keys():
                    tpms72.append(expression[label][ensembl])
            tpm72 = numpy.median(tpms72)
            top = numpy.max([tpm0, tpm48, tpm72])
            
            #
            # filter 2 --- identify fold-changes using discrete values
            #        
            ###
            ###            [round(x, epsilon)/epsilon ] + 1
            ###  FC = abs  -------------------------------- > 1
            ###            [round(y, epsilon)/epsilon ] + 1
            ###
            ###
            ###  epsilon = 1
            sam48 = numpy.around(tpm48) + 1
            sam72 = numpy.around(tpm72) + 1
            ref = numpy.around(tpm0) + 1
            
            fc48 = sam48/ref; fc72 = sam72/ref
            log2FC48 = numpy.log2(fc48); log2FC72 = numpy.log2(fc72)
            
            #
            # filter 3 --- noise
            #
            tpms0_int = numpy.around(tpms0) + 1
            rsem_tpms0 = numpy.std(tpms0_int) / (numpy.sqrt(len(tpms0_int)) * numpy.mean(tpms0_int))
            
            tpms48_int = numpy.around(tpms48) + 1
            rsem_tpms48 = numpy.std(tpms48_int) / (numpy.sqrt(len(tpms48_int)) * numpy.mean(tpms48_int))
            
            tpms72_int = numpy.around(tpms72) + 1
            rsem_tpms72 = numpy.std(tpms72_int) / (numpy.sqrt(len(tpms72_int)) * numpy.mean(tpms72_int))
            noise = numpy.mean([rsem_tpms0, rsem_tpms48, rsem_tpms72])
            
            #
            # selection
            #
            if top < expression_threshold:
                including = False
                info = '\t\t\t WARNING: low-expression gene discarded. Expression changes from {:.3f} to {:.3f} and {:.3f}'.format(tpm0, tpm48, tpm72)
                print(info)
                
            if (including == True):
                # check that trend is consistent in both timepoints
                if (log2FC48*log2FC72) < 0:
                    including = False
                    info = '\t\t\t WARNING: inconsistent trend: {:.3f} and {:.3f}'.format(log2FC48, log2FC72)
                    print(info)
                # check that you are abs log2FC > 1 at both conditions
                if numpy.max([numpy.abs(log2FC48), numpy.abs(log2FC72)]) < 1:
                    including = False
                    info = '\t\t\t WARNING: small change gene discarded: Expressions: {:.2f}, {:.2f}, {:.2f}. Fold changes: {:.2f}, {:.2f}'.format(tpm0, tpm48, tpm72, log2FC48, log2FC72)
                    print(info)
                    
            if (including == True) and (noise > noise_threshold):
                including = False
                info = '\t\t\t WARNING: noisy gene discarded.'
                print(info)
            
            #
            # store DEGs that passed filters
            #
            if including == True:
                print('\t\t\t SUCCESS: expression: [{:.3f}, {:.3f}, {:.3f}]; log2FCs: [{:.2f}, {:.2f}]'.format(tpm0, tpm48, tpm72, log2FC48, log2FC72))
                
                accumulative_expression_difference = numpy.log10(numpy.abs(tpm72-tpm0) + numpy.abs(tpm48-tpm0))
                accumulative_fold_change = log2FC48 + log2FC72
                
                box = [ensembl, gene_name, biotype, description, '{:.2f}'.format(tpm0), '{:.2f}'.format(tpm48), '{:.2f}'.format(tpm72), '{:.3f}'.format(log2FC48), '{:.3f}'.format(log2FC72), '{:.3f}'.format(accumulative_expression_difference), '{:.3f}'.format(accumulative_fold_change)]
                container.append(box)
                
        #
        # info about selection
        #
        print('\t pased filters {}\n'.format(len(container)))
        
        #
        # store results
        #
        results_table_file = results_table_dir + mouse + '_' + trend + '.tsv'
        f = open(results_table_file, 'w')
        f.write('ENSEMBL\tGene name\tBiotype\tDescription\tTPM @ T0\tTPM @ T48\tTPM @T72\tlog2FC @ T48\tlog2TPM @ T72\tAccumulative expression difference (log10 TPM)\tAccumulative log2FC\n')
        for box in container:
            info = '\t'.join(box)
            info = info + '\n'
            f.write(info)
        f.close()        

working with mouse a3922 and trend up
	 detected 854 DEGs
	 passed tests 854
		 working with Gm20431
		 working with Rsc1a1
		 working with Slco4a1
		 working with Cd248
		 working with Clvs1
			 SUCCESS: expression: [0.139, 0.771, 2.628]; log2FCs: [1.00, 2.00]
		 working with Gm5741
			 SUCCESS: expression: [0.000, 0.920, 4.738]; log2FCs: [1.00, 2.58]
		 working with Adamts14
		 working with Psg17
		 working with Ccdc194
		 working with Aldh1a1
		 working with P2ry2
			 SUCCESS: expression: [0.185, 2.407, 3.678]; log2FCs: [1.58, 2.32]
		 working with Slc38a3
		 working with Arhgap22
		 working with Ramp3
		 working with Upp1
		 working with Hoxb8
		 working with Mbp
		 working with Vmn2r27
		 working with Cntnap5c
		 working with Rgs8
		 working with Aldh1a7
			 SUCCESS: expression: [0.156, 2.119, 1.833]; log2FCs: [1.58, 1.58]
		 working with Gimap9
			 SUCCESS: expression: [0.161, 1.605, 2.066]; log2FCs: [1.58, 1.58]
		 working with Parvg
		 working with Coro2a
			 SUCCESS: expressio

			 SUCCESS: expression: [1.688, 6.743, 6.533]; log2FCs: [1.42, 1.42]
		 working with Recql4
			 SUCCESS: expression: [0.986, 2.857, 4.000]; log2FCs: [1.00, 1.32]
		 working with Ifi27
			 SUCCESS: expression: [6.620, 25.442, 25.235]; log2FCs: [1.70, 1.70]
		 working with Akap6
		 working with Mis18bp1
			 SUCCESS: expression: [3.011, 9.674, 12.004]; log2FCs: [1.46, 1.70]
		 working with Mest
			 SUCCESS: expression: [3.351, 10.992, 13.370]; log2FCs: [1.58, 1.81]
		 working with Jag1
		 working with Trip13
			 SUCCESS: expression: [3.684, 13.514, 12.889]; log2FCs: [1.58, 1.49]
		 working with Adamts4
			 SUCCESS: expression: [3.192, 11.534, 12.186]; log2FCs: [1.70, 1.70]
		 working with Plaur
		 working with Cenpn
			 SUCCESS: expression: [3.872, 14.134, 13.455]; log2FCs: [1.58, 1.49]
		 working with Gbp3
			 SUCCESS: expression: [3.631, 14.780, 12.754]; log2FCs: [1.68, 1.49]
		 working with Lmnb1
			 SUCCESS: expression: [22.286, 82.715, 82.615]; log2FCs: [1.87, 1.87]
		 working with 

		 working with Tecrl
			 SUCCESS: expression: [4.641, 8.181, 10.641]; log2FCs: [0.58, 1.00]
		 working with Egln3
		 working with Shmt2
			 SUCCESS: expression: [46.956, 108.835, 121.614]; log2FCs: [1.20, 1.36]
		 working with Wdr17
		 working with Hmgcr
			 SUCCESS: expression: [45.962, 108.653, 115.936]; log2FCs: [1.23, 1.32]
		 working with Tspan32
		 working with Mx2
			 SUCCESS: expression: [2.811, 4.197, 7.791]; log2FCs: [0.32, 1.17]
		 working with Dna2
			 SUCCESS: expression: [3.350, 9.814, 8.268]; log2FCs: [1.46, 1.17]
		 working with Zdhhc2
		 working with Micall2
			 SUCCESS: expression: [3.037, 5.908, 7.185]; log2FCs: [0.81, 1.00]
		 working with Rad18
			 SUCCESS: expression: [6.241, 16.166, 16.750]; log2FCs: [1.28, 1.36]
		 working with Alg8
			 SUCCESS: expression: [7.061, 17.217, 17.549]; log2FCs: [1.17, 1.25]
		 working with Pold1
			 SUCCESS: expression: [14.362, 32.481, 33.487]; log2FCs: [1.14, 1.18]
		 working with Cenps
			 SUCCESS: expression: [7.563, 14.513, 18

			 SUCCESS: expression: [211.242, 40.303, 52.542]; log2FCs: [-2.37, -1.97]
		 working with 1700029J07Rik
		 working with Gm10548
		 working with Slc24a3
			 SUCCESS: expression: [10.827, 2.340, 2.645]; log2FCs: [-2.00, -1.58]
		 working with Fam83f
		 working with Lca5l
		 working with Eef1a2
		 working with Gm30302
			 SUCCESS: expression: [2.606, 0.268, 0.700]; log2FCs: [-2.00, -1.00]
		 working with Slc18a1
		 working with Scrg1
			 SUCCESS: expression: [10.867, 2.503, 3.408]; log2FCs: [-1.58, -1.58]
		 working with Pcsk2
		 working with Negr1
		 working with Osmr
		 working with Cpne4
		 working with Zdhhc23
		 working with Galnt17
		 working with Pde1b
		 working with Txnip
			 SUCCESS: expression: [313.874, 85.298, 87.122]; log2FCs: [-1.87, -1.84]
		 working with Mcoln2
		 working with Casc1
		 working with Greb1l
		 working with Rasl10b
			 SUCCESS: expression: [6.710, 2.831, 2.002]; log2FCs: [-1.00, -1.42]
		 working with Fat4
		 working with Pik3ip1
			 SUCCESS: expression: [

		 working with Nr1h3
		 working with Gm48270
		 working with Ech1
			 SUCCESS: expression: [136.067, 60.547, 69.108]; log2FCs: [-1.14, -0.97]
		 working with Rwdd3
		 working with Calcoco1
			 SUCCESS: expression: [160.096, 66.653, 79.610]; log2FCs: [-1.24, -0.99]
		 working with Stk36
			 SUCCESS: expression: [7.015, 3.418, 2.998]; log2FCs: [-1.00, -1.00]
		 working with Dennd5b
			 SUCCESS: expression: [43.085, 20.429, 21.450]; log2FCs: [-1.07, -1.00]
		 working with Ppm1l
			 SUCCESS: expression: [8.026, 3.438, 3.631]; log2FCs: [-1.17, -0.85]
		 working with Wnk2
			 SUCCESS: expression: [10.266, 3.846, 5.065]; log2FCs: [-1.14, -0.87]
		 working with Klc3
			 SUCCESS: expression: [2.552, 1.082, 1.175]; log2FCs: [-1.00, -1.00]
		 working with H2-T23
			 SUCCESS: expression: [12.224, 3.957, 5.886]; log2FCs: [-1.38, -0.89]
		 working with Ecm1
			 SUCCESS: expression: [31.407, 11.896, 15.191]; log2FCs: [-1.30, -1.00]
		 working with Stpg1
			 SUCCESS: expression: [8.230, 2.827, 4.237]

...

		 working with Tex15
		 working with H1f3
		 working with Akap12
			 SUCCESS: expression: [19.444, 61.336, 46.664]; log2FCs: [1.63, 1.26]
		 working with Gm15807
		 working with H3c3
			 SUCCESS: expression: [0.942, 3.349, 2.905]; log2FCs: [1.00, 1.00]
		 working with Fancd2
			 SUCCESS: expression: [3.598, 8.090, 9.709]; log2FCs: [0.85, 1.14]
		 working with Cep295nl
		 working with Net1
			 SUCCESS: expression: [3.184, 8.634, 9.285]; log2FCs: [1.32, 1.32]
		 working with Egfl7
			 SUCCESS: expression: [13.915, 39.556, 34.125]; log2FCs: [1.45, 1.22]
		 working with Nt5dc2
			 SUCCESS: expression: [12.633, 28.766, 30.161]; log2FCs: [1.10, 1.15]
		 working with Ttyh1
			 SUCCESS: expression: [0.733, 3.324, 1.955]; log2FCs: [1.00, 0.58]
		 working with Spag1
			 SUCCESS: expression: [2.155, 3.346, 4.843]; log2FCs: [0.42, 1.00]
		 working with Gm18329
			 SUCCESS: expression: [2.454, 4.427, 6.058]; log2FCs: [0.74, 1.22]
		 working with Tk1
			 SUCCESS: expression: [28.715, 64.625, 70.407

		 working with H3c15
			 SUCCESS: expression: [10.506, 5.259, 3.677]; log2FCs: [-1.00, -1.26]
		 working with Kcnma1
		 working with Lratd2
			 SUCCESS: expression: [20.525, 8.450, 7.923]; log2FCs: [-1.29, -1.29]
		 working with Tmem202
			 SUCCESS: expression: [4.612, 2.690, 1.773]; log2FCs: [-0.58, -1.00]
		 working with Gm5117
		 working with Maf
			 SUCCESS: expression: [9.510, 5.009, 3.846]; log2FCs: [-0.87, -1.14]
		 working with Rprm
		 working with Rpl7a-ps9
		 working with Trnp1
			 SUCCESS: expression: [5.393, 3.298, 2.175]; log2FCs: [-0.58, -1.00]
		 working with Tmem191c
			 SUCCESS: expression: [33.160, 11.823, 13.050]; log2FCs: [-1.39, -1.28]
		 working with Eya1
			 SUCCESS: expression: [19.091, 7.687, 7.896]; log2FCs: [-1.15, -1.15]
		 working with Igsf8
			 SUCCESS: expression: [326.058, 158.853, 140.126]; log2FCs: [-1.03, -1.21]
		 working with Zfp784
		 working with Scn1b
			 SUCCESS: expression: [32.607, 26.221, 14.352]; log2FCs: [-0.33, -1.18]
		 working with Aph1

			 SUCCESS: expression: [5.258, 18.780, 31.663]; log2FCs: [1.74, 2.46]
		 working with Emp1
		 working with Tlx2
		 working with Cntd1
		 working with Myorg
		 working with Hspb1
		 working with Oas1c
			 SUCCESS: expression: [0.350, 2.555, 1.588]; log2FCs: [2.00, 1.58]
		 working with Mgat3
			 SUCCESS: expression: [1.277, 6.288, 6.882]; log2FCs: [1.81, 2.00]
		 working with Axdnd1
		 working with Vmn1r227
		 working with Vmn1r73
		 working with Me3
		 working with Pwwp3b
		 working with Tgtp2
		 working with Hoxb9
			 SUCCESS: expression: [1.433, 7.369, 7.570]; log2FCs: [2.00, 2.17]
		 working with Chrm3
		 working with Dlx6
		 working with Gm13394
		 working with Nid2
			 SUCCESS: expression: [3.219, 14.789, 19.326]; log2FCs: [2.00, 2.32]
		 working with Mfrp
		 working with Cavin2
			 SUCCESS: expression: [0.458, 2.196, 2.525]; log2FCs: [1.58, 2.00]
		 working with Tbx5
		 working with Rnf207
		 working with Fam184b
			 SUCCESS: expression: [0.556, 2.825, 3.029]; log2FCs: [1.00, 1

			 SUCCESS: expression: [16.766, 52.055, 48.096]; log2FCs: [1.56, 1.44]
		 working with Pycr1
			 SUCCESS: expression: [3.168, 9.315, 10.031]; log2FCs: [1.32, 1.46]
		 working with Gsdme
			 SUCCESS: expression: [0.892, 2.661, 2.810]; log2FCs: [1.00, 1.00]
		 working with Cdca3
			 SUCCESS: expression: [14.940, 31.744, 40.495]; log2FCs: [1.04, 1.36]
		 working with Gm35060
		 working with Myc
			 SUCCESS: expression: [31.251, 61.442, 88.685]; log2FCs: [0.95, 1.49]
		 working with Eme1
			 SUCCESS: expression: [1.401, 2.900, 3.617]; log2FCs: [1.00, 1.32]
		 working with Atp10d
		 working with Emb
		 working with Xkr5
		 working with Myrf
		 working with Cntn1
		 working with Pask
			 SUCCESS: expression: [1.223, 4.716, 3.154]; log2FCs: [1.58, 1.00]
		 working with Dnm1
			 SUCCESS: expression: [2.847, 9.392, 7.681]; log2FCs: [1.32, 1.17]
		 working with Pbk
			 SUCCESS: expression: [9.387, 24.183, 27.719]; log2FCs: [1.32, 1.54]
		 working with Shank1
		 working with Sema6a
			 SUCCESS:

		 working with Acta2
		 working with Mreg
			 SUCCESS: expression: [78.524, 30.489, 24.200]; log2FCs: [-1.37, -1.68]
		 working with Irx3
			 SUCCESS: expression: [40.052, 16.503, 12.292]; log2FCs: [-1.19, -1.66]
		 working with 1110002E22Rik
		 working with Eps8
			 SUCCESS: expression: [699.480, 226.012, 221.751]; log2FCs: [-1.62, -1.65]
		 working with Spsb1
			 SUCCESS: expression: [37.428, 14.243, 11.414]; log2FCs: [-1.34, -1.66]
		 working with Dkk3
			 SUCCESS: expression: [721.354, 330.273, 231.287]; log2FCs: [-1.13, -1.64]
		 working with Nrp1
			 SUCCESS: expression: [51.226, 18.775, 15.822]; log2FCs: [-1.38, -1.61]
		 working with Glt28d2
		 working with Kazn
			 SUCCESS: expression: [21.213, 8.978, 7.030]; log2FCs: [-1.14, -1.46]
		 working with Piwil2
		 working with Chchd10
			 SUCCESS: expression: [72.350, 38.388, 22.698]; log2FCs: [-0.90, -1.60]
		 working with Nedd4l
			 SUCCESS: expression: [86.269, 31.519, 27.168]; log2FCs: [-1.40, -1.64]
		 working with Rhof
		 wor